## This script loads the current model and performs an evaluation of it

### Initialize
First, initialize the model with all parameters


In [1]:
from data_source import DataSource
from visualize import Visualize
from sphere import Sphere
from model import Model
from loss import TripletLoss, ImprovedTripletLoss
from training_set import TrainingSet
from average_meter import AverageMeter
from data_splitter import DataSplitter
from mission_indices import MissionIndices
from database_parser import DatabaseParser

import torch
import torch.nn.functional as F
import torchvision
import torchvision.transforms as transforms
from torch.autograd import Variable
from torch.utils.tensorboard import SummaryWriter
from torchsummary import summary

import pyshtools
from pyshtools import spectralanalysis
from pyshtools import shio
from pyshtools import expand

import sys
import time
import math
import operator
import numpy as np
import pandas as pd
import open3d as o3d
import matplotlib.pyplot as plt
from matplotlib.backends.backend_pdf import PdfPages

from tqdm.auto import tqdm
import scipy.stats as st
from scipy import spatial

%reload_ext autoreload
%autoreload 2

In [2]:
torch.cuda.set_device(0)
torch.backends.cudnn.benchmark = True
n_features = 2
bandwidth = 100
from model_relu_old import ModelOld
net = ModelOld(n_features, bandwidth).cuda()
restore = False
optimizer = torch.optim.SGD(net.parameters(), lr=5e-3, momentum=0.9)
batch_size = 12
num_workers = 12
descriptor_size = 256
net_input_size = 2*bandwidth
cache = 50
criterion = ImprovedTripletLoss(margin=2, alpha=0.5, margin2=0.2)
writer = SummaryWriter()
stored_model = './net_params_arche_low_res_small_lidar_only.pkl'
net.load_state_dict(torch.load(stored_model))
#summary(net, input_size=[(2, 200, 200), (2, 200, 200), (2, 200, 200)])

<All keys matched successfully>

Initialize the data source

In [3]:
#dataset_path = "/media/scratch/berlukas/spherical/"
dataset_path = "/home/berlukas/data/arche_low_res2/"
db_parser = DatabaseParser(dataset_path)

training_missions, test_missions = MissionIndices.get_arche_low_res()
training_indices, test_indices = db_parser.extract_training_and_test_indices(
    training_missions, test_missions)
print(f'Found {len(test_missions)} test indices.')

n_test_data = 2500
n_test_cache = n_test_data
ds_test = DataSource(dataset_path, n_test_cache, -1)
idx = np.array(test_indices['idx'].tolist())
ds_test.load(n_test_data, idx, filter_clusters=True)
n_test_data = len(ds_test.anchors)

Reading missions db from /home/berlukas/data/arche_low_res2/missions.csv
Read 21253 entries.



Found 2 test indices.
Loading anchors from:	/home/berlukas/data/arche_low_res2//training_anchor_pointclouds/ and /home/berlukas/data/arche_low_res2//training_anchor_sph_images/



Loading positives from:	/home/berlukas/data/arche_low_res2//training_positive_pointclouds/ and /home/berlukas/data/arche_low_res2//training_positive_sph_images/



Loading negatives from:	/home/berlukas/data/arche_low_res2//training_negative_pointclouds/ and /home/berlukas/data/arche_low_res2//training_negative_sph_images/



Done loading dataset.
	Anchor point clouds total: 	125
	Anchor images total: 		125
	Anchor poses total: 		125
	Positive point clouds total: 	125
	Positive images total: 		125
	Positive poses total: 		125
	Negative point clouds total: 	125
	Negative images total: 		125
	Negative poses total: 		125


In [4]:
ds_test.rotate_all_positives('z', 20)

In [6]:
test_set = TrainingSet(restore, bandwidth)
test_set.generateAll(ds_test)

Generating features from 0 to 125
Generating anchor spheres



Processing time in total 29.14858341217041 for 125 anchors.
Generating positive spheres



Processing time in total 57.29148817062378 for 250 positives.
Generating negative spheres



Processing time in total 87.14081144332886 for 375 negatives.
Generated all pcl features
Processing time in total 87.14081144332886 for 375 items.
Processing time avg is 0.23238


In [7]:
# hack for removing the images
test_set.anchor_features = test_set.anchor_features[:,0:2,:,:]
test_set.positive_features = test_set.positive_features[:,0:2,:,:]
test_set.negative_features = test_set.negative_features[:,0:2,:,:]


n_test_set = len(test_set)
print("Total size: ", n_test_set)
test_loader = torch.utils.data.DataLoader(test_set, batch_size=10, shuffle=False, num_workers=1, pin_memory=True, drop_last=False)

Total size:  125


  ## Generate the descriptors for anchor and positive

In [8]:
def accuracy(dista, distb):
    margin = 0
    pred = (dista - distb - margin).cpu().data
    acc = ((pred < 0).sum()).float()/dista.size(0)
    return acc

net.eval()
n_iter = 0
anchor_embeddings = np.empty(1)
positive_embeddings = np.empty(1)
with torch.no_grad():
    test_accs = AverageMeter()
    test_pos_dist = AverageMeter()
    test_neg_dist = AverageMeter()

    for batch_idx, (data1, data2, data3) in enumerate(test_loader):
        embedded_a, embedded_p, embedded_n = net(data1.cuda().float(), data2.cuda().float(), data3.cuda().float())
        dist_to_pos, dist_to_neg, loss, loss_total = criterion(embedded_a, embedded_p, embedded_n)
        writer.add_scalar('Ext_Test/Loss', loss, n_iter)

        acc = accuracy(dist_to_pos, dist_to_neg)
        test_accs.update(acc, data1.size(0))
        test_pos_dist.update(dist_to_pos.cpu().data.numpy().sum())
        test_neg_dist.update(dist_to_neg.cpu().data.numpy().sum())

        writer.add_scalar('Ext_Test/Accuracy', test_accs.avg, n_iter)
        writer.add_scalar('Ext_Test/Distance/Positive', test_pos_dist.avg, n_iter)
        writer.add_scalar('Ext_Test/Distance/Negative', test_neg_dist.avg, n_iter)

        anchor_embeddings = np.append(anchor_embeddings, embedded_a.cpu().data.numpy().reshape([1,-1]))
        positive_embeddings = np.append(positive_embeddings, embedded_p.cpu().data.numpy().reshape([1,-1]))
        n_iter = n_iter + 1
        
desc_anchors = anchor_embeddings[1:].reshape([n_test_set, descriptor_size])
desc_positives = positive_embeddings[1:].reshape([n_test_set, descriptor_size])      

## Simple old testing pipeline (index based)

In [29]:
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_positives)
p_norm = 2
max_pos_dist = 0.05
max_anchor_dist = 1
for n_nearest_neighbors in tqdm(range(1,21)):
    pos_count = 0
    anchor_count = 0
    idx_count = 0
    for idx in range(n_test_set):
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices

        for nn_i in nn_indices:
            if (nn_i >= n_test_set):
                break;
            dist = spatial.distance.euclidean(desc_positives[nn_i,:], desc_positives[idx,:])
            if (dist <= max_pos_dist):
                pos_count = pos_count + 1;
                break
        for nn_i in nn_indices:
            if (nn_i >= n_test_set):
                break;
            dist = spatial.distance.euclidean(desc_positives[nn_i,:], desc_anchors[idx,:])
            if (dist <= max_anchor_dist):
                anchor_count = anchor_count + 1;
                break
        for nn_i in nn_indices:
            if (nn_i == idx):
                idx_count = idx_count + 1;
                break
    pos_precision = (pos_count*1.0) / n_test_set
    anchor_precision = (anchor_count*1.0) / n_test_set
    idx_precision = (idx_count*1.0) / n_test_set
    
    print(f'recall {idx_precision} for {n_nearest_neighbors} neighbors')
    writer.add_scalar('Ext_Test/Precision/Positive_Distance', pos_precision, n_nearest_neighbors)
    writer.add_scalar('Ext_Test/Precision/Anchor_Distance', anchor_precision, n_nearest_neighbors)
    writer.add_scalar('Ext_Test/Precision/Index_Count', idx_precision, n_nearest_neighbors)

recall 0.022222222222222223 for 1 neighbors
recall 0.044444444444444446 for 2 neighbors
recall 0.06666666666666667 for 3 neighbors
recall 0.08888888888888889 for 4 neighbors
recall 0.1111111111111111 for 5 neighbors
recall 0.13333333333333333 for 6 neighbors
recall 0.15555555555555556 for 7 neighbors
recall 0.17777777777777778 for 8 neighbors
recall 0.2 for 9 neighbors
recall 0.2222222222222222 for 10 neighbors
recall 0.24444444444444444 for 11 neighbors
recall 0.26666666666666666 for 12 neighbors
recall 0.28888888888888886 for 13 neighbors
recall 0.3111111111111111 for 14 neighbors
recall 0.3333333333333333 for 15 neighbors
recall 0.35555555555555557 for 16 neighbors
recall 0.37777777777777777 for 17 neighbors
recall 0.4 for 18 neighbors
recall 0.4222222222222222 for 19 neighbors
recall 0.4444444444444444 for 20 neighbors



## New testing pipeline (location based)

In [9]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_positives)
p_norm = 2
max_pos_dist = 5.0
max_anchor_dist = 1
anchor_poses = ds_test.anchor_poses
positive_poses = ds_test.positive_poses
assert len(anchor_poses) == len(positive_poses)

for n_nearest_neighbors in tqdm(range(1,21)):    
    loc_count = 0
    for idx in range(n_test_set):
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices

        for nn_i in nn_indices:
            if (nn_i >= n_test_set):
                break;
            dist = spatial.distance.euclidean(positive_poses[nn_i,5:8], anchor_poses[idx,5:8])
            if (dist <= max_pos_dist):
                loc_count = loc_count + 1;
                break
                
    loc_precision = (loc_count*1.0) / n_test_set    
    print(f'recall {loc_precision} for {n_nearest_neighbors} neighbors')
    #print(f'{loc_precision}')
    #writer.add_scalar('Ext_Test/Precision/Location', loc_precision, n_nearest_neighbors)    

Running test pipeline for a map size of 125 descriptors.


recall 0.96 for 1 neighbors
recall 0.984 for 2 neighbors
recall 0.992 for 3 neighbors
recall 0.992 for 4 neighbors
recall 0.992 for 5 neighbors
recall 0.992 for 6 neighbors
recall 0.992 for 7 neighbors
recall 0.992 for 8 neighbors
recall 0.992 for 9 neighbors
recall 0.992 for 10 neighbors
recall 0.992 for 11 neighbors
recall 0.992 for 12 neighbors
recall 0.992 for 13 neighbors
recall 0.992 for 14 neighbors
recall 0.992 for 15 neighbors
recall 0.992 for 16 neighbors
recall 0.992 for 17 neighbors
recall 0.992 for 18 neighbors
recall 0.992 for 19 neighbors
recall 1.0 for 20 neighbors



## Place Voting using Global Spectral Analysis


In [23]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_positives)
p_norm = 2
max_pos_dist = 5.0

anchor_poses = ds_test.anchor_poses
anchor_clouds = ds_test.anchors
anchor_features = test_set.anchor_features

positive_poses = ds_test.positive_poses
positive_clouds = ds_test.positives
positive_features = test_set.anchor_features

for n_nearest_neighbors in tqdm(range(1,21)):        
    n_matches = 0
    loc_count = 0    
    for idx in range(0, n_test_set):        
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices
        
        z_scores = [0] * n_nearest_neighbors
        contains_match = False        
        true_match_idx = 0
        for i in range(0, n_nearest_neighbors):
            nn_i = nn_indices[i]            
            if (nn_i >= n_test_set):
                print(f'ERROR: index {nn_i} is outside of {n_data}')
                break;
                
            dist = spatial.distance.euclidean(positive_poses[nn_i,5:8], anchor_poses[idx,5:8])
            if (dist <= max_pos_dist):
                contains_match = True                
                true_match_idx = i
                
            a_range = anchor_features[idx][0,:,:]
            p_range = positive_features[nn_i][0,:,:]
            a_intensity = anchor_features[idx][1,:,:]
            p_intensity = positive_features[nn_i][1,:,:]
            #a_img = anchor_features[idx][2,:,:]
            #p_img = positive_features[nn_i][2,:,:]
                                               
            a_range_coeffs = pyshtools.expand.SHExpandDH(a_range, sampling=1)
            p_range_coeffs = pyshtools.expand.SHExpandDH(p_range, sampling=1)
            
            a_intensity_coeffs = pyshtools.expand.SHExpandDH(a_intensity, sampling=1)
            p_intensity_coeffs = pyshtools.expand.SHExpandDH(p_intensity, sampling=1)
            
            #a_img_coeffs = pyshtools.expand.SHExpandDH(a_img, sampling=1)
            #p_img_coeffs = pyshtools.expand.SHExpandDH(p_img, sampling=1)
            
            #a_fused = np.empty([3, a_range_coeffs.shape[0], a_range_coeffs.shape[1]])
            #p_fused = np.empty([3, p_range_coeffs.shape[0], p_range_coeffs.shape[1]])
            #print(a_range_coeffs.shape)
            #a_fused[0,:] = a_range_coeffs
            
            
            admit, error, corr = spectralanalysis.SHAdmitCorr(a_range_coeffs, p_range_coeffs)            
            for l in range(0, 4):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores[i] = z_scores[i] + score
                #if math.isinf(z_scores[i]):
                    #print(f'z-score is inf: prob = {prob}, z-score {st.norm.ppf(1-(1-prob)/2)}')
            
        #if (contains_match is not True):
            #print(f'Match not found for index {idx} and {n_nearest_neighbors} neighbors')
            #continue
        
        n_matches = n_matches + 1
        max_index, max_z_score = max(enumerate(z_scores), key=operator.itemgetter(1))
        matching_index = nn_indices[max_index]
        dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], anchor_poses[idx,5:8])
        if (dist <= max_pos_dist):
            loc_count = loc_count + 1;            
        else:
            #print(f'Place invalid: distance anchor <-> positive: {dist} with score {max_z_score}.')            
            matching_index = nn_indices[true_match_idx]
            dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], positive_poses[true_match_idx,5:8])
            #print(f'Distance positive <-> true_match: {dist}, true_match score: {z_scores[true_match_idx]}')
                
    loc_precision = (loc_count*1.0) / n_matches    
    #print(f'Recall {loc_precision} for {n_nearest_neighbors} neighbors with {n_matches}/{n_data} correct matches.')
    print(f'{loc_precision}')
    writer.add_scalar('Ext_Test/Precision/Voting', loc_precision, n_nearest_neighbors)

Running test pipeline for a map size of 271 descriptors.


0.9040590405904059
0.8966789667896679
0.9188191881918819
0.9188191881918819
0.922509225092251


KeyboardInterrupt: 

## Place Voting using Global Spectral Analysis


In [9]:
print(f'Running test pipeline for a map size of {len(desc_positives)} descriptors.')
sys.setrecursionlimit(50000)
tree = spatial.KDTree(desc_positives)
p_norm = 2
max_pos_dist = 5.0

anchor_poses = ds_test.anchor_poses
anchor_clouds = ds_test.anchors
anchor_features = test_set.anchor_features

positive_poses = ds_test.positive_poses
positive_clouds = ds_test.positives
positive_features = test_set.anchor_features

for n_nearest_neighbors in tqdm(range(1,21)):        
    n_matches = 0
    loc_count = 0    
    final_count = 0
    for idx in range(0, n_test_set):        
        nn_dists, nn_indices = tree.query(desc_anchors[idx,:], p = p_norm, k = n_nearest_neighbors)
        nn_indices = [nn_indices] if n_nearest_neighbors == 1 else nn_indices
        
        z_scores_range = [0] * n_nearest_neighbors
        z_scores_intensity = [0] * n_nearest_neighbors
        z_scores_image = [0] * n_nearest_neighbors
        contains_match = False        
        true_match_idx = 0
        for i in range(0, n_nearest_neighbors):
            nn_i = nn_indices[i]            
            if (nn_i >= n_test_set):
                print(f'ERROR: index {nn_i} is outside of {n_data}')
                break;
                
            dist = spatial.distance.euclidean(positive_poses[nn_i,5:8], anchor_poses[idx,5:8])
            if (dist <= max_pos_dist):
                contains_match = True                
                true_match_idx = i
                
            a_range = anchor_features[idx][0,:,:]
            p_range = positive_features[nn_i][0,:,:]
            a_intensity = anchor_features[idx][1,:,:]
            p_intensity = positive_features[nn_i][1,:,:]
            #a_img = anchor_features[idx][2,:,:]
            #p_img = positive_features[nn_i][2,:,:]
                                                           
            a_range_coeffs = pyshtools.expand.SHExpandDH(a_range, sampling=1)
            p_range_coeffs = pyshtools.expand.SHExpandDH(p_range, sampling=1)
            
            a_intensity_coeffs = pyshtools.expand.SHExpandDH(a_intensity, sampling=1)
            p_intensity_coeffs = pyshtools.expand.SHExpandDH(p_intensity, sampling=1)
            
            #a_img_coeffs = pyshtools.expand.SHExpandDH(a_img, sampling=1)
            #p_img_coeffs = pyshtools.expand.SHExpandDH(p_img, sampling=1)
            
            tapers, eigenvalues, taper_order = spectralanalysis.SHReturnTapers(2.01, 1)
            saa_range = spectralanalysis.spectrum(a_range_coeffs)            
            saa_intensity = spectralanalysis.spectrum(a_intensity_coeffs)    
            #saa_img = spectralanalysis.spectrum(a_img_coeffs)    
            saa = np.empty([n_features, saa_range.shape[0]])
            saa[0,:] = saa_range
            saa[1,:] = saa_intensity
            #saa[2,:] = saa_img
            #saa = np.mean(saa, axis=0)
            saa = np.amax(saa, axis=0)
            
            spp_range = spectralanalysis.spectrum(p_range_coeffs)            
            spp_intensity = spectralanalysis.spectrum(p_intensity_coeffs)    
            #spp_img = spectralanalysis.spectrum(p_img_coeffs)    
            spp = np.empty([n_features, spp_range.shape[0]])
            spp[0,:] = saa_range
            spp[1,:] = saa_intensity
            #spp[2,:] = saa_img
            #spp = np.mean(spp, axis=0)
            spp = np.amax(spp, axis=0)
            
            sap_range = spectralanalysis.cross_spectrum(a_range_coeffs, p_range_coeffs)            
            sap_intensity = spectralanalysis.cross_spectrum(a_intensity_coeffs, p_intensity_coeffs)    
            #sap_img = spectralanalysis.cross_spectrum(a_img_coeffs, p_img_coeffs)    
            sap = np.empty([n_features, sap_range.shape[0]])
            sap[0,:] = saa_range
            sap[1,:] = saa_intensity
            #sap[2,:] = saa_img
            #sap = np.mean(sap, axis=0)
            sap = np.amax(sap, axis=0)
            
            #saa = spectralanalysis.spectrum(a_coeffs)
            #spp = spectralanalysis.spectrum(p_coeffs)
            #sap = spectralanalysis.cross_spectrum(a_coeffs, p_coeffs)
            
            #admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_img, saa_img, spp_img, tapers)            
            #admit, corr = spectralanalysis.SHBiasAdmitCorr(sap, saa, spp, tapers)
            
            admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_range, saa_range, spp_range, tapers)                        
            for l in range(0, 10):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores_range[i] = z_scores_range[i] + score
                            
            admit, corr = spectralanalysis.SHBiasAdmitCorr(sap_intensity, saa_intensity, spp_intensity, tapers)                            
            for l in range(0, 10):                
                prob = spectralanalysis.SHConfidence(l, corr[l])                
                score = st.norm.ppf(1-(1-prob)/2) if prob < 0.99 else 4.0
                z_scores_intensity[i] = z_scores_intensity[i] + score                           
            
        #if (contains_match is not True):
            #print(f'Match not found for index {idx} and {n_nearest_neighbors} neighbors')
            #continue
        
        n_matches = n_matches + 1
        max_index_range, max_z_score_range = max(enumerate(z_scores_range), key=operator.itemgetter(1))
        max_index_intensity, max_z_score_intensity = max(enumerate(z_scores_intensity), key=operator.itemgetter(1))
        
        #print(f'max range: {max_z_score_range}, max intensity: {max_z_score_intensity}')
        max_index = max_index_range if max_z_score_range > max_z_score_intensity else max_index_intensity
        matching_index = nn_indices[max_index]
        dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], anchor_poses[idx,5:8])
        if (dist <= max_pos_dist):
            loc_count = loc_count + 1;            
        else:
            #print(f'Place invalid: distance anchor <-> positive: {dist} with score {max_z_score}.')            
            matching_index = nn_indices[true_match_idx]
            dist = spatial.distance.euclidean(positive_poses[matching_index,5:8], positive_poses[true_match_idx,5:8])
            #print(f'Distance positive <-> true_match: {dist}, true_match score: {z_scores[true_match_idx]}')
                
    loc_precision = (loc_count*1.0) / n_matches    
    #print(f'Recall {loc_precision} for {n_nearest_neighbors} neighbors with {n_matches}/{n_data} correct matches.')
    print(f'{loc_precision}')
    writer.add_scalar('Ext_Test/Precision/WindowedVoting', loc_precision, n_nearest_neighbors)

Running test pipeline for a map size of 271 descriptors.


0.8487084870848709
0.8560885608856088
0.8782287822878229
0.8856088560885609
0.9003690036900369
0.915129151291513
0.9040590405904059
0.9188191881918819
0.9261992619926199
0.933579335793358
0.9261992619926199
0.9261992619926199
0.933579335793358
0.9298892988929889
0.9298892988929889
0.9372693726937269
0.9372693726937269
0.940959409594096
0.9446494464944649
0.9446494464944649



In [ ]:
anchor_poses[0:100,5:8]

In [35]:
a = np.matrix('1 2; 3 4')
b = np.matrix('4 5; 6 7')
c = np.empty([2,2,2])
c[0,:,:] = a
c[1,:,:] = b
np.mean(c, axis=0)
a.shape[0]

2